In [1]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Se conecta con la BD MySQL AWS

In [2]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Genera un df con el archivo `csv` "CPTEVENTS"

In [8]:
df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\CPTEVENTS.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,subject_id,hadm_id,icustay_id,starttime,endtime,itemid,amount,amountuom,rate,...,totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,comments_editedby,comments_canceledby,comments_date,originalamount,originalrate
0,118897,42367,139932,250305,2147-10-29 16:45:00,2147-10-29 16:46:00,225799,60.000000,ml,NaN,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,60.0,60.0
1,118898,42367,139932,250305,2147-10-20 13:17:00,2147-10-20 13:18:00,223258,10.000000,units,NaN,...,NaN,0,0,1,Rewritten,NaN,RN,2147-10-20 13:18:00,10.0,10.0
2,118899,42367,139932,250305,2147-10-29 03:23:00,2147-10-29 03:53:00,226089,99.999999,ml,199.999998,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,100.0,200.0


In [4]:
df = df.replace(np.nan, 0)

### Inserta los registros del df "CPTEVENTS" en la BD

In [5]:
sql_insert = ''' INSERT 
                   INTO  CPTEVENTS
                         (row_id, subject_id, hadm_id, costcenter, chartdate, cpt_cd, cpt_number, cpt_suffix, ticket_id_seq, 
                          sectionheader, subsectionheader, description)
                 VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) '''
num_reng = 0
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    subj = int(df.loc[i, 'subject_id'])
    hadm = int(df.loc[i, 'hadm_id'])
    coce = df.loc[i, 'costcenter']
    chdt = df.loc[i, 'chartdate']
    cptc = df.loc[i, 'cpt_cd']
    cptn = int(df.loc[i, 'cpt_number'])
    cpts = df.loc[i, 'cpt_suffix']
    tkid = int(df.loc[i, 'ticket_id_seq'])
    sehd = df.loc[i, 'sectionheader']
    sbhd = df.loc[i, 'subsectionheader']
    desc = df.loc[i, 'description']
    reg_s = (row, subj, hadm, coce, chdt, cptc, cptn, cpts, tkid, sehd, sbhd, desc)
    cursor.execute(sql_insert, reg_s)
    num_reng += 1
    if num_reng == 100:
        con.commit()
        num_reng = 0

### Confirma la inserción de los registros a la BD

In [6]:
con.commit()

### Cierra el cursor y la conexion a la BD

In [7]:
cursor.close()
con.close()